In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
print(f"Training data:{training_data}")

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
print(f"Test data:{test_data}")

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
print(f"train_dataloader:{train_dataloader}")

Training data:Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Test data:Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()
train_dataloader:<torch.utils.data.dataloader.DataLoader object at 0x74dca0176e10>


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Softmax(),
            nn.Linear(512, 512),
            nn.Softmax(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model.to(device)
dir(nn)

['AdaptiveAvgPool1d',
 'AdaptiveAvgPool2d',
 'AdaptiveAvgPool3d',
 'AdaptiveLogSoftmaxWithLoss',
 'AdaptiveMaxPool1d',
 'AdaptiveMaxPool2d',
 'AdaptiveMaxPool3d',
 'AlphaDropout',
 'AvgPool1d',
 'AvgPool2d',
 'AvgPool3d',
 'BCELoss',
 'BCEWithLogitsLoss',
 'BatchNorm1d',
 'BatchNorm2d',
 'BatchNorm3d',
 'Bilinear',
 'CELU',
 'CTCLoss',
 'ConstantPad1d',
 'ConstantPad2d',
 'ConstantPad3d',
 'Container',
 'Conv1d',
 'Conv2d',
 'Conv3d',
 'ConvTranspose1d',
 'ConvTranspose2d',
 'ConvTranspose3d',
 'CosineEmbeddingLoss',
 'CosineSimilarity',
 'CrossEntropyLoss',
 'CrossMapLRN2d',
 'DataParallel',
 'Dropout',
 'Dropout2d',
 'Dropout3d',
 'ELU',
 'Embedding',
 'EmbeddingBag',
 'FeatureAlphaDropout',
 'Flatten',
 'Fold',
 'FractionalMaxPool2d',
 'FractionalMaxPool3d',
 'GLU',
 'GRU',
 'GRUCell',
 'GroupNorm',
 'Hardshrink',
 'Hardtanh',
 'HingeEmbeddingLoss',
 'Identity',
 'InstanceNorm1d',
 'InstanceNorm2d',
 'InstanceNorm3d',
 'KLDivLoss',
 'L1Loss',
 'LPPool1d',
 'LPPool2d',
 'LSTM',
 'LST

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    del X
    del y


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        del X
        del y
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
import torch
import gc
class ClearCache:
    def __enter__(self):
        gc.collect()
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#epochs = 10
# Use the context manager
with ClearCache():
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
        test_loop(test_dataloader, model, loss_fn)
    del model
print("Done!")

Epoch 1
-------------------------------
loss: 2.302898  [   64/60000]


/opt/anaconda/envs/wmlce/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 2.303435  [ 6464/60000]
loss: 2.292202  [12864/60000]
loss: 2.265998  [19264/60000]
loss: 2.198264  [25664/60000]
loss: 2.121951  [32064/60000]
loss: 2.005985  [38464/60000]
loss: 1.922505  [44864/60000]
loss: 1.794803  [51264/60000]
loss: 1.714407  [57664/60000]
Test Error: 
 Accuracy: 37.2%, Avg loss: 1.687562 

Epoch 2
-------------------------------
loss: 1.647642  [   64/60000]
loss: 1.632145  [ 6464/60000]
loss: 1.536112  [12864/60000]
loss: 1.523247  [19264/60000]
loss: 1.472647  [25664/60000]
loss: 1.481865  [32064/60000]
loss: 1.460976  [38464/60000]
loss: 1.490685  [44864/60000]
loss: 1.363291  [51264/60000]
loss: 1.244262  [57664/60000]
Test Error: 
 Accuracy: 40.9%, Avg loss: 1.297257 

Epoch 3
-------------------------------
loss: 1.240235  [   64/60000]
loss: 1.293821  [ 6464/60000]
loss: 1.190833  [12864/60000]
loss: 1.175418  [19264/60000]
loss: 1.149427  [25664/60000]
loss: 1.255291  [32064/60000]
loss: 1.174854  [38464/60000]
loss: 1.334683  [44864/60000]
loss: 

In [ ]:
from google.colab import files

filename = list(files.upload().keys())[0]

Saving bhuv.jpg to bhuv.jpg


In [ ]:
import cv2
import numpy as np
import open3d as o3d
from mmpose.apis import inference_top_down_pose_model, init_pose_model
import mmcv
def detect_keypoints(image_path):
    image = cv2.imread(image_path)
    config_file = 'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w32_coco_256x192.py'
    checkpoint_file = 'checkpoints/hrnet_w32_coco_256x192-c78dce93_20200708.pth'
    model = init_pose_model(config_file, checkpoint_file, device='cuda:0')

    image_path = 'bhuv.jpg'
    image = mmcv.imread(image_path)

    result = inference_top_down_pose_model(model, image)
    model.show_result(image, result, 'result.jpg')

    keypoints = np.random.rand(18, 2) * image.shape[1]
    return keypoints

In [ ]:
def estimate_depth(image_path):
    image = cv2.imread(image_path)
    depth_map = np.random.rand(image.shape[0], image.shape[1]) * 10  # Random depth map for demonstration
    return depth_map

def reconstruct_3d(keypoints, depth_map):
    point_cloud = []
    for i, keypoint in enumerate(keypoints):
        x, y = int(keypoint[0]), int(keypoint[1])
        z = depth_map[y, x]
        point_cloud.append([x, y, z])
    point_cloud = np.array(point_cloud)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)

    o3d.visualization.draw_geometries([pcd])

image_path = 'input_image.jpg'
keypoints = detect_keypoints(image_path)
depth_map = estimate_depth(image_path)
reconstruct_3d(keypoints, depth_map)


In [ ]:
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=9b7f1523063b624f442a0bd64ef91415041b62fe365840fdc6330e79f32ba0c5
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=73ecf0c468631cf2d63d8e010be4331ea5d0149cfb92fa2302a1d6446a7de383
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu121_pyt221/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 138.6 MB/s eta 0:00:00
